In [10]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import os

In [15]:
def calibrate():
    pathToImages = 'camera_cal/'
    calFiles = [f for f in os.listdir(pathToImages) if os.path.isfile(join(pathToImages, f))]
    images = glob.glob('camera_cal/calibration*.jpg')
    
    # prepare object points
    nx = 9
    ny = 6
    
    objpoints = []
    imgpoints = []
    objp = np.zeros((ny * nx, 3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)

    # Make a list of calibration images
    for image in images:
        img = mpimg.imread(image)

        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        # If found, draw corners
        if ret == True:
            # Draw and display the corners
#             cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
#             plt.imshow(img)
#             plt.show()
            imgpoints.append(corners)
            objpoints.append(objp)
            
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return mtx, dist


In [16]:
mtx, dist = calibrate()

NameError: name 'join' is not defined

In [ ]:
def getThresholdedSobel(undist, orient='x', thresh_min=20, thresh_max=100):
    gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
    
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    elif orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    else:
        return None
    
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    return binary_output

In [ ]:
def mag_thresh(undist, sobel_kernel=3, mag_thresh=(100, 200)):
    # Convert to grayscale
    gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx ** 2 + sobely ** 2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag) / 255 
    gradmag = (gradmag / scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

In [ ]:
# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

In [ ]:
def getGradientImgCombined(undist):
    ksize = 3

    gradx = getThresholdedSobel(undist, orient='x')
    grady = getThresholdedSobel(undist, orient='y')

    # plt.imshow(sobelx, cmap='gray')
    # plt.show()
    # plt.imshow(sobely, cmap='gray')
    # plt.show()

    magn = mag_thresh(undist)
    direction = dir_threshold(undist, sobel_kernel=ksize, thresh=(0, np.pi/2))

    combined = np.zeros_like(direction)
    combined[((gradx == 1) & (grady == 1)) | ((magn == 1) & (direction == 1))] = 1
    return combined

In [ ]:
def getColorCombined(undist, 
                     gray_thresh=(180, 255), 
                     r_thresh=(230, 255)):
#     gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
#     gray_binary = np.zeros_like(gray)
#     gray_binary[(gray > gray_thresh[0]) & (gray <= gray_thresh[1])] = 1
    
    R = undist[:,:,0]
    r_binary = np.zeros_like(R)
    r_binary[(R > r_thresh[0]) & (R <= r_thresh[1])] = 1
    
    hls = cv2.cvtColor(undist, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    
    thresh = (90, 255)
    s_binary = np.zeros_like(S)
    s_binary[(S > thresh[0]) & (S <= thresh[1])] = 1
    
    combined = np.zeros_like(r_binary)
#     combined[((gray_binary == 1) | (r_binary == 1))] = 1
#     combined[((r_binary == 1) & (s_binary == 1))] = 1
    combined[((s_binary == 1))] = 1
#     combined[((r_binary == 1))] = 1
    
    return combined

In [ ]:
def getWarped(img):
    img_size = (img.shape[1], img.shape[0])
    
    src = np.float32([(590, 450), (690, 450), (1060, 690), (250, 690)])
    dst = np.float32([(250, 0), (1060, 0), (1060, 690), (250, 690)])

#     plt.imshow(img, cmap='gray')
#     for point in src:
#         plt.plot(point[0], point[1], '.')
#     plt.show()
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
#     plt.imshow(warped, cmap='gray')
#     for point in dst:
#         plt.plot(point[0], point[1], '.')
#     plt.show()
    
    return warped

# img = mpimg.imread('test_images/straight_lines1.jpg')
img = mpimg.imread('test_images/test4.jpg')
undist = cv2.undistort(img, mtx, dist, None, mtx)
# plt.imshow(undist)
# plt.show()

grad_combined = getGradientImgCombined(undist)
# plt.imshow(grad_combined, cmap='gray')
# plt.show()

color_combined = getColorCombined(undist, gray_thresh=(190, 255), r_thresh=(210, 255))

# Combine the two binary thresholds
combined_binary = np.zeros_like(grad_combined)
combined_binary[(grad_combined == 1) | (color_combined == 1)] = 1

# plt.imshow(combined_binary, cmap='gray')
# plt.show()

warped = getWarped(combined_binary)
plt.imshow(warped, cmap='gray')
plt.show()

histogram = np.sum(warped[warped.shape[0]//2:,:], axis=0)
plt.plot(histogram)
plt.show()